# Text Classification

## Aim: To implemnet text classification using Naive bayes and SVM
## Dataset: Collection of tweets having 4 classes namely:
  * Politics
  * Entertainment
  * Sports
  * Medical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install advertools -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.9/272.9 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install contractions -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install clean-text -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import advertools as adv
import re
import contractions
from cleantext import clean
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
from sklearn import preprocessing
from sklearn import svm

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/text_classification_dataset.xlsx')
df.head()

,text,type
0,@ACNI2012 @TheToka920 Never knew having 1 or 2...,sports
1,"MYCA Magical Moments:\n\nSeptember, 2011: Sham...",sports
2,The current state of last year's @BBL finalist...,sports
3,@HOLLYJISOO Why did you bring a cricket...,sports
4,Babar Azam only Pakistani included in the ICC ...,sports


In [ ]:
df['type'].value_counts()

politics         345
medical          299
entertainment    260
sports           258
Name: type, dtype: int64

In [ ]:
class Preprocess:

  def __init__(self,text):
    self.ltext=text.lower()
    self.text=contractions.fix(self.ltext)
    self.stop_words=set(adv.stopwords['english'])

  def emoji(self):
    e_txt=clean(self.text, no_emoji=True)
    return e_txt

  def remove_links(self,text):
    text  = re.sub(r"https\S+", "", text)
    return text

  def remove_numerics(self,text):
    self.n_text = re.sub("[^a-zA-Z0-9]", " ", text)
    return self.n_text

  def remove_quotes_brackets(self,text):
    self.text1=re.sub("[\"\'\(\)]", "", text)
    return self.text1

  def remove_punctuations(self,text):
    text1=re.sub(r"[^\w\s]", " ",text)
    return text1

  def tokenize(self,text):
    self.t_text=word_tokenize(text)
    return self.t_text

  def stop_word(self,text):
    self.sw=[]
    for i in text:
      if i not in self.stop_words:
        self.sw.append(i)
    return self.sw

  def lemmatize(self,text):
    lemma =WordNetLemmatizer()
    l_text = [lemma.lemmatize(word) for word in text]
    return l_text

  def preprocess(self):
    e_txt=self.emoji()
    link_txt=self.remove_links(e_txt)
    n_txt=self.remove_numerics(link_txt)
    q_text=self.remove_quotes_brackets(n_txt)
    p_text=self.remove_punctuations(q_text)
    t_text=self.tokenize(p_text)
    l_text=self.lemmatize(t_text)
    swtext=self.stop_word(l_text)
    word=' '.join(swtext)
    return word


In [ ]:
swl=[]
for i in df['text']:
  p=Preprocess(i)
  sw=p.preprocess()
  swl.append(sw)
df['preprocess_text']=swl

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df['type']= label_encoder.fit_transform(df['type'])

In [ ]:
df['type'].value_counts()

2    345
1    299
0    260
3    258
Name: type, dtype: int64

In [ ]:
x=df['text']
y=df['type']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [ ]:
tf=TfidfVectorizer(analyzer='word',stop_words='english')

In [ ]:
x_train_tf=tf.fit_transform(x_train)
x_train_tf=tf.transform(x_train)

In [ ]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(x_train_tf, y_train)

MultinomialNB()

In [ ]:
x_test_tf = tf.transform(x_test)

In [ ]:
y_pred = naive_bayes_classifier.predict(x_test_tf)

In [ ]:
print(classification_report(y_test, y_pred,target_names=['entertainment','medical','politics','sports']))

               precision    recall  f1-score   support

entertainment       0.82      0.76      0.79        72
      medical       0.80      0.88      0.84        78
     politics       0.78      0.87      0.82       112
       sports       0.93      0.77      0.84        87

     accuracy                           0.83       349
    macro avg       0.83      0.82      0.82       349
 weighted avg       0.83      0.83      0.83       349



In [ ]:
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train_tf,y_train)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(x_test_tf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  84.81375358166189


In [ ]:
print(classification_report(y_test, predictions_SVM,target_names=['entertainment','medical','politics','sports']))

               precision    recall  f1-score   support

entertainment       0.86      0.79      0.83        72
      medical       0.89      0.79      0.84        78
     politics       0.76      0.94      0.84       112
       sports       0.97      0.83      0.89        87

     accuracy                           0.85       349
    macro avg       0.87      0.84      0.85       349
 weighted avg       0.86      0.85      0.85       349



## Conclusion: SVM classifiers gives a better accuracy compared to Naive Bayes, therefore it is considered to be the best classifier for this data